In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_path = '/content/drive/MyDrive/data/cats and dogs/train'
test_path = '/content/drive/MyDrive/data/cats and dogs/validation'

In [4]:
train_data_gen = ImageDataGenerator(rescale=1./255,shear_range=0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [5]:
training_set = train_data_gen.flow_from_directory(train_path,target_size=(64,64),
                                                  batch_size = 4,
                                                  class_mode = 'binary')

test_set = test_data_gen.flow_from_directory(test_path,target_size=(64,64),
                                             batch_size=4,class_mode='binary')

Found 53 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [6]:
## creating convolution network
# 1st convolution layer
cnn = Sequential()
cnn.add(Conv2D(filters=32,padding='same',activation='relu',
               kernel_size=(3,3),input_shape=[64,64,3],strides = (2,2)))

cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [7]:
# 2nd convolution layer
cnn.add(Conv2D(filters=32,padding='same',kernel_size=(3,3),
               activation='relu',strides=(2,2)))
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [8]:
# faltten the layers
cnn.add(Flatten())

## fully connected layers
cnn.add(Dense(units=64,activation='relu'))


In [9]:
# output layers and apply svm at output layers
cnn.add(Dense(units=1,kernel_regularizer=tf.keras.regularizers.l2(0.01),
              activation='linear')) # for binary classification always use linear activation

In [ ]:
## compiling the cnn model
cnn.compile(optimizer='adam',loss = 'hinge',metrics=['accuracy']) # for binary classification using svm loss is hinge fro multiclass loss is squared_hinge
fit  = cnn.fit(x = training_set,validation_data=test_set,epochs = 100)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(fit.history['loss'],label = 'train_loss')
plt.plot(fit.history['val_loss'],label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(fit.history['accuracy'],label = 'train_acc')
plt.plot(fit.history['val_accuracy'],label = 'val_acc')
plt.legend()
plt.show()

In [15]:
#save file
from tensorflow.keras.models import load_model
cnn.save('model_cat_dig_svm.h5')
model = load_model('model_cat_dig_svm.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [16]:
##making single prediction
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/ANN/cat_0_1090.jpeg',target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image,axis = 0)
result = cnn.predict(test_image)


In [19]:
if result[0]<0:
  print('this is a cat')
else:
  print('ths is a dog')

this is a cat
